In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import tensorflow as tf

/Users/Ryan/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Will implement [conv-relu-pool]xN -> [affine]xM -> [softmax or SVM] and write script to print accuracy of models on test and train data

# Preprocess the Data

In [3]:
data = pd.read_csv("./data/handwritten_data_785.csv", encoding = "utf8")

In [4]:
np.random.seed(0)
data = data.values
np.random.shuffle(data)
X, y = data[:,1:], data[:,0]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Data Dimensions

In [19]:
print(X_train.shape)
print(X_test.shape)

#images are 28x28
img_size = 28
img_size_flat = img_size * img_size
img_shape = (img_size, img_size)

num_channels = 1
num_classes = 26

X_train = X_train.reshape(X_train.shape[0], img_size, img_size, 1)
X_test = X_test.reshape(X_test.shape[0], img_size, img_size, 1)

(297629, 28, 28, 1)
(74408, 28, 28, 1)


In [20]:
mean_image = np.mean(X_train, axis=0).astype(np.int64)
X_train = (X_train - mean_image)/255
X_test = (X_test - mean_image)/255

# Helper Functions

In [22]:
def create_biases(length):
    return tf.Variable(tf.constant(0.01, shape=[length]))
'''
One problem is that the distribution of the outputs from a randomly initialized 
neuron has a variance that grows with the number of inputs. It turns out that we can normalize the variance 
of each neuron’s output to 1 by scaling its weight vector by the square root of its fan-in (i.e. its number of inputs).
That is, the recommended heuristic is to initialize each neuron’s weight vector as: w = np.random.randn(n) / sqrt(n), 
where n is the number of its inputs. This ensures that all neurons in the network initially have approximately the 
same output distribution and empirically improves the rate of convergence.
'''
# xavier_initializer is 2/sqrt(n) when uniform=False
def create_weights(shape):
    W = tf.get_variable("W", shape=shape,
           initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    return W

def new_conv_layer(input, input_channels, filter_size, num_filters, x_stride=1, y_stride=1, use_pooling=True, use_batchnorm=False):
    '''
    input (4d Tensor):
        -Image number.
        -Y-axis of each image.
        -X-axis of each image.
        -Channels of each image.
    input_channels: number of channels in the input
    filter_size: window size of filter (ex: 5x5)
    num_filters: number of filters to use
    x_stride: amount to move filter over on x axis
    y_stride: amount to move filter over on y axis
        ex: strides=[1, 2, 2, 1] would mean that the filter
        is moved 2 pixels across the x- and y-axis of the image
    use_pooling: whether or not to downsample image resolution using 2x2 max-pooling
    use_batchnorm: whether or not to use batch normalization on this layer
    '''
    weights_shape = [filter_size, filter_size, input_channels, num_filters]
    weights = create_weights(weights_shape)
    
    biases = create_biases(num_filters)
    
    #create layer
    layer = tf.nn.conv2d(input, weights, strides=[1,x_stride,y_stride,1], padding='VALID') + biases
    
    # ** maybe split pooling and batch_norm??
    layer = tf.nn.relu(layer)
    
    if use_pooling:
        pool_1 = tf.nn.max_pool(layer, ksize=[1,2,2,1], strides=[1, 2, 2, 1], padding='VALID')
    